In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/LDS0/Topic_11_Agoda/demo'

/content/gdrive/MyDrive/LDS0/Topic_11_Agoda/demo


## Tiền xử lý dữ liệu tiếng Việt

In [ ]:
# Có thể dùng thư viện underthesea hoặc bất cứ thư viện xử lý tiếng Việt nào khác

In [ ]:
!pip install underthesea
!pip install demoji
!pip install pyvi

In [ ]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import regex
import demoji
from pyvi import ViPosTagger, ViTokenizer
import string

In [ ]:
# https://underthesea.readthedocs.io/en/v1.1.5/readme.html
# https://github.com/undertheseanlp/underthesea

In [ ]:
##LOAD EMOJICON
file = open('files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
file.close()
#################
#LOAD TEENCODE
file = open('files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('files/english-vnmese.txt', 'r', encoding="utf8")
english_lst = file.read().split('\n')
english_dict = {}
for line in english_lst:
    key, value = line.split('\t')
    english_dict[key] = str(value)
file.close()
################
#LOAD wrong words
file = open('files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

In [ ]:
def process_text(text, emoji_dict, teen_dict, wrong_lst):
    document = text.lower()
    document = document.replace("’",'')
    document = regex.sub(r'\.+', ".", document)
    new_sentence =''
    for sentence in sent_tokenize(document):
        # if not(sentence.isascii()):
        ###### CONVERT EMOJICON
        sentence = ''.join(emoji_dict[word]+' ' if word in emoji_dict else word for word in list(sentence))
        ###### CONVERT TEENCODE
        sentence = ' '.join(teen_dict[word] if word in teen_dict else word for word in sentence.split())
        ###### DEL Punctuation & Numbers
        pattern = r'(?i)\b[a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]+\b'
        sentence = ' '.join(regex.findall(pattern,sentence))
        # ...
        ###### DEL wrong words
        sentence = ' '.join('' if word in wrong_lst else word for word in sentence.split())
        new_sentence = new_sentence+ sentence + '. '
    document = new_sentence
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    #...
    return document

In [ ]:
example = '''Vc mình ở đây 2 đêm và rất hài lòng về ks .
MT này nằm ở vị trí đắc địa, ngay trước biển và ngay khu phố trung tâm, nhộn nhịp và dễ đi lại.
Về phòng ốc thì mình ở phòng view thành phố nhưng do săn đc giá tốt nên cũng rất hài lònggggggg 💚💚💚. Phòng sạch sẽ, tiện nghi và thoải mái, đặc biệt rất là thích ghế sofa để relax trong phòng.
Nhân viên thật sự chu đáo và nhẹ nhàng. Dù không phải là khách sạn cực hiện đại và lộng lẫy nhưng xét về tổng thể với ks 4 sao thì rất hài lòng!'''

In [ ]:
print(example)

Vc mình ở đây 2 đêm và rất hài lòng về ks . 
MT này nằm ở vị trí đắc địa, ngay trước biển và ngay khu phố trung tâm, nhộn nhịp và dễ đi lại. 
Về phòng ốc thì mình ở phòng view thành phố nhưng do săn đc giá tốt nên cũng rất hài lònggggggg 💚💚💚. Phòng sạch sẽ, tiện nghi và thoải mái, đặc biệt rất là thích ghế sofa để relax trong phòng. 
Nhân viên thật sự chu đáo và nhẹ nhàng. Dù không phải là khách sạn cực hiện đại và lộng lẫy nhưng xét về tổng thể với ks 4 sao thì rất hài lòng!


In [ ]:
document = process_text(example, emoji_dict, teen_dict, wrong_lst)
document

'vợ chồng mình ở đây đêm và rất hài lòng về khách sạn. này nằm ở vị trí đắc địa trước biển và phố tâm nhộn nhịp và dễ đi lại. về phòng ốc thì mình ở phòng cảnh thành phố nhưng săn được giá tốt nên cũng rất hài lònggggggg yêu yêu yêu. phòng sạch sẽ tiện và thoải mái đặc biệt rất là thích ghế để phòng. nhân viên thật sự đáo và nhẹ nhàng. dù không phải là khách sạn cực hiện đại và lộng lẫy nhưng xét về tổng thể với khách sạn thì rất hài lòng.'

In [ ]:
# Chuẩn hóa unicode tiếng việt
def loaddicchar():
    uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    dicchar = loaddicchar()
    return regex.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
document = covert_unicode(document)
document

'vợ chồng mình ở đây đêm và rất hài lòng về khách sạn. này nằm ở vị trí đắc địa trước biển và phố tâm nhộn nhịp và dễ đi lại. về phòng ốc thì mình ở phòng cảnh thành phố nhưng săn được giá tốt nên cũng rất hài lònggggggg yêu yêu yêu. phòng sạch sẽ tiện và thoải mái đặc biệt rất là thích ghế để phòng. nhân viên thật sự đáo và nhẹ nhàng. dù không phải là khách sạn cực hiện đại và lộng lẫy nhưng xét về tổng thể với khách sạn thì rất hài lòng.'

In [ ]:
def process_special_word(text):
    # có thể có nhiều từ đặc biệt cần ráp lại với nhau
    new_text = ''
    text_lst = text.split()
    i= 0
    # không, chẳng, chả...
    if 'không' in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
            #print(word)
            #print(i)
            if  word == 'không':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
    else:
        new_text = text
    return new_text.strip()

In [ ]:
x = process_special_word(document)
x

'vợ chồng mình ở đây đêm và rất hài lòng về khách sạn. này nằm ở vị trí đắc địa trước biển và phố tâm nhộn nhịp và dễ đi lại. về phòng ốc thì mình ở phòng cảnh thành phố nhưng săn được giá tốt nên cũng rất hài lònggggggg yêu yêu yêu. phòng sạch sẽ tiện và thoải mái đặc biệt rất là thích ghế để phòng. nhân viên thật sự đáo và nhẹ nhàng. dù không_phải là khách sạn cực hiện đại và lộng lẫy nhưng xét về tổng thể với khách sạn thì rất hài lòng.'

In [ ]:
import re
# Hàm để chuẩn hóa các từ có ký tự lặp
def normalize_repeated_characters(text):
    # Thay thế mọi ký tự lặp liên tiếp bằng một ký tự đó
    # Ví dụ: "lònggggg" thành "lòng", "thiệtttt" thành "thiệt"
    return re.sub(r'(.)\1+', r'\1', text)

# Áp dụng hàm chuẩn hóa cho văn bản
# print(normalize_repeated_characters(example))

In [ ]:
document = normalize_repeated_characters(document)
print(document)

vợ chồng mình ở đây đêm và rất hài lòng về khách sạn. này nằm ở vị trí đắc địa trước biển và phố tâm nhộn nhịp và dễ đi lại. về phòng ốc thì mình ở phòng cảnh thành phố nhưng săn được giá tốt nên cũng rất hài lòng yêu yêu yêu. phòng sạch sẽ tiện và thoải mái đặc biệt rất là thích ghế để phòng. nhân viên thật sự đáo và nhẹ nhàng. dù không phải là khách sạn cực hiện đại và lộng lẫy nhưng xét về tổng thể với khách sạn thì rất hài lòng.


In [ ]:
def process_postag_thesea(text):
    new_document = ''
    for sentence in sent_tokenize(text):
        sentence = sentence.replace('.','')
        ###### POS tag
        lst_word_type = ['N','Np','A','AB','V','VB','VY','R']
        # lst_word_type = ['A','AB','V','VB','VY','R']
        sentence = ' '.join( word[0] if word[1].upper() in lst_word_type else '' for word in pos_tag(process_special_word(word_tokenize(sentence, format="text"))))
        new_document = new_document + sentence + ' '
    ###### DEL excess blank space
    new_document = regex.sub(r'\s+', ' ', new_document).strip()
    return new_document

In [ ]:
document = process_postag_thesea(document)

In [ ]:
document

'vợ_chồng đêm rất hài_lòng nằm vị_trí đắc_địa biển phố_tâm nhộn_nhịp dễ phòng_ốc phòng cảnh săn được_giá tốt cũng rất hài_lòng yêu_yêu yêu phòng sạch_sẽ tiện thoải_mái đặc_biệt rất là thích ghế phòng nhân_viên thật_sự_đáo không_phải là khách_sạn cực hiện_đại lộng_lẫy xét tổng_thể khách_sạn rất hài_lòng'

In [ ]:
def remove_stopword(text, stopwords):
    ###### REMOVE stop words
    document = ' '.join('' if word in stopwords else word for word in text.split())
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [ ]:
document = remove_stopword(document, stopwords_lst)
document

'vợ_chồng đêm hài_lòng nằm đắc_địa biển phố_tâm nhộn_nhịp phòng_ốc phòng cảnh săn được_giá tốt hài_lòng yêu_yêu yêu phòng sạch_sẽ tiện thoải_mái thích ghế phòng thật_sự_đáo khách_sạn cực hiện_đại lộng_lẫy xét tổng_thể khách_sạn hài_lòng'

## Tạo thêm các cột mới dựa trên việc đếm từ/icon positive và negative
- Tạo danh sách các từ positive -> lưu vào file positive_VN.txt (mỗi từ trên 1 dòng)
- Tạo danh sách các từ negative -> lưu vào file negative_VN.txt (mỗi từ trên 1 dòng)
- Tạo danh sách các positve emojis -> lưu vào file positive_emoji.txt (mỗi icon trên 1 dòng)
- Tạo danh sách các negative emojis -> lưu vào file negative_emoji.txt (mỗi icon trên 1 dòng)
- Đọc vào các list tương ứng
- Viết function để đọc 1 chuỗi -> đếm số lượng positive words/ emojis hoặc  negative words/ ememojis, danh sách từ kết quả
- Tạo ra 2 cột mới cho dataframe: negative_count/ positive_count
- ...

In [ ]:
positive_words = [
    "thích", "tốt", "xuất sắc", "tuyệt vời", "tuyệt hảo", "đẹp", "ổn", "ngon",
    "hài lòng", "ưng ý", "hoàn hảo", "chất lượng", "thú vị", "nhanh",
    "tiện lợi", "dễ sử dụng", "hiệu quả", "ấn tượng",
    "nổi bật", "tận hưởng", "tốn ít thời gian", "thân thiện", "hấp dẫn",
    "cao cấp", "độc đáo", "rất tốt", "rất thích", "tận tâm", "đáng tin cậy", "đẳng cấp",
    "hấp dẫn", "an tâm", "thúc đẩy", "cảm động", "phục vụ tốt", "làm hài lòng", "gây ấn tượng", "nổi trội",
    "sáng tạo", "phù hợp", "tận tâm", "hiếm có", "cải thiện", "hoà nhã", "chăm chỉ", "cẩn thận",
    "vui vẻ", "sáng sủa", "hào hứng", "đam mê", "vừa vặn", "đáng tiền"
]

In [ ]:
len(positive_words)

52

In [ ]:
negative_words = [
    "kém", "tệ", "buồn", "chán",
    "kém chất lượng", "không thích", "không ổn",
    "không hợp", "không đáng tin cậy", "không chuyên nghiệp",
    "không phản hồi", "không an toàn", "không phù hợp", "không thân thiện", "không linh hoạt", "không đáng giá",
    "không ấn tượng", "không tốt", "chậm", "khó khăn", "phức tạp",
    "khó chịu", "gây khó dễ", "rườm rà", "thất bại", "tồi tệ", "khó xử", "không thể chấp nhận", "tồi tệ","không rõ ràng",
    "không chắc chắn", "rối rắm", "không tiện lợi", "không đáng tiền", "chưa đẹp", "không đẹp",
    'tồi', 'xấu', 'không hài lòng', 'bẩn', 'khó chịu', 'không sạch sẽ', 'không thoải mái', 'không đáng', 'quá tệ', 'rất tệ',
    'thất vọng', 'chán', 'tệ hại', 'kinh khủng', 'khủng khiếp', 'không ưng ý'
]

In [ ]:
len(negative_words)

52

In [ ]:
def find_words(document, list_of_words):
    document_lower = document.lower()
    word_count = 0
    word_list = []

    for word in list_of_words:
        if word in document_lower:
            print(word)
            word_count += document_lower.count(word)
            word_list.append(word)

    return word_count, word_list

In [ ]:
# Example usage
document = '''Vc mình ở đây 2 đêm và rất hài lòng về ks .
MT này nằm ở vị trí đắc địa, ngay trước biển và ngay khu phố trung tâm, nhộn nhịp và dễ đi lại.
Về phòng ốc thì mình ở phòng view thành phố nhưng do săn đc giá tốt nên cũng rất hài lònggggggg 💚💚💚. Phòng sạch sẽ, tiện nghi và thoải mái, đặc biệt rất là thích ghế sofa để relax trong phòng.
Nhân viên thật sự chu đáo và nhẹ nhàng. Dù không phải là khách sạn cực hiện đại và lộng lẫy nhưng xét về tổng thể với ks 4 sao thì rất hài lòng!'''

In [ ]:
document = normalize_repeated_characters(document)

In [ ]:
positive_count, positive_word_list = find_words(document, positive_words)

print("Number of positive words:", positive_count)
print("List of positive words:", positive_word_list)

thích
tốt
ổn
hài lòng
Number of positive words: 6
List of positive words: ['thích', 'tốt', 'ổn', 'hài lòng']


In [ ]:
negative_count, negative_word_list = find_words(document, negative_words)

print("Number of negative words:", negative_count)
print("List of negative words:", negative_word_list)

Number of negative words: 0
List of negative words: []


In [ ]:
# Count emojis positive and negative
negative_emojis = [
    "😞", "😔", "🙁", "☹️", "😕",
    "😢", "😭", "😖", "😣", "😩",
    "😠", "😡", "🤬", "😤", "😰",
    "😨", "😱", "😪", "😓", "🥺",
    "😒", "🙄", "😑", "😬", "😶",
    "🤯", "😳", "🤢", "🤮", "🤕",
    "🥴", "🤔", "😷", "🙅‍♂️", "🙅‍♀️",
    "🙆‍♂️", "🙆‍♀️", "🙇‍♂️", "🙇‍♀️", "🤦‍♂️",
    "🤦‍♀️", "🤷‍♂️", "🤷‍♀️", "🤢", "🤧",
    "🤨", "🤫", "👎", "👊", "✊", "🤛", "🤜",
    "🤚", "🖕"
]

In [ ]:
len(negative_emojis)

54

In [ ]:
positive_emojis = [
    "😄", "😃", "😀", "😁", "😆",
    "😅", "🤣", "😂", "🙂", "🙃",
    "😉", "😊", "😇", "🥰", "😍",
    "🤩", "😘", "😗", "😚", "😙",
    "😋", "😛", "😜", "🤪", "😝",
    "🤗", "🤭", "🥳", "😌", "😎",
    "🤓", "🧐", "👍", "🤝", "🙌", "👏", "👋",
    "🤙", "✋", "🖐️", "👌", "🤞",
    "✌️", "🤟", "👈", "👉", "👆",
    "👇", "☝️", "💚"
]

In [ ]:
len(positive_emojis)

50

In [ ]:
positive_icon, positive_icon_list = find_words(document, positive_emojis)

print("Number of positive icons:", positive_icon)
print("List of positive icons:", positive_icon_list)

💚
Number of positive icons: 1
List of positive icons: ['💚']


In [ ]:
negative_icon, negative_icon_list = find_words(document, negative_emojis)

print("Number of negative words:", negative_icon)
print("List of negative words:", negative_icon_list)

Number of negative words: 0
List of negative words: []


In [ ]:
document_1 = '''Thang máy thường xuyên tắc trong giờ sáng, tôi phải chờ 10 phút mới xuống được sảnh, thang rất bí.
Bể bơi bẩn, toàn hóa chất 😡😡. Bù lại vị trí đẹp, gần biển 👍. Sẽ quay lại nếu cải thiện.'''

In [ ]:
positive_count, positive_word_list = find_words(document_1, positive_words)

print("Number of positive words:", positive_count)
print("List of positive words:", positive_word_list)

đẹp
cải thiện
Number of positive words: 2
List of positive words: ['đẹp', 'cải thiện']


In [ ]:
negative_count, negative_word_list = find_words(document_1, negative_words)

print("Number of negative words:", negative_count)
print("List of negative words:", negative_word_list)

bẩn
Number of negative words: 1
List of negative words: ['bẩn']


In [ ]:
positive_icon, positive_icon_list = find_words(document_1, positive_emojis)

print("Number of positive icons:", positive_icon)
print("List of positive icons:", positive_icon_list)

👍
Number of positive icons: 1
List of positive icons: ['👍']


In [ ]:
negative_icon, negative_icon_list = find_words(document_1, negative_emojis)

print("Number of negative words:", negative_icon)
print("List of negative words:", negative_icon_list)

😡
Number of negative words: 2
List of negative words: ['😡']
